# Import model

In [77]:
from ultralytics import YOLO

model = YOLO('./generic_logo_model/detect/train/weights/best.pt')

In [78]:
results = model('/Users/nickjohnson/olm_data/olm_pics/', conf=0.05, iou=0, stream=True)

# Create an embedding matrix of all predictions

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import numpy as np
import torch
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from imgbeddings import imgbeddings
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO

# Initialize the imgbeddings model
ibed = imgbeddings()
vgg_model = VGG16(include_top=False, pooling='avg')

def load_image(img_input):
    if isinstance(img_input, str):
        return Image.open(img_input).convert('RGB')
    elif isinstance(img_input, np.ndarray):  
        return Image.fromarray(img_input).convert('RGB')

def generate_embeddings(image_paths, ibed, vgg_model):
    embeddings = []
    for image_path in image_paths:
        # Load image
        image = load_image(image_path)
        
        # Process for imgbeddings
        ibed_embedding = ibed.to_embeddings(image)
        ibed_embedding = ibed_embedding.flatten()

        # Process for VGG16
        vgg_image = image.resize((224, 224))
        vgg_array = preprocess_input(np.expand_dims(np.array(vgg_image), axis=0))
        with torch.no_grad():
            vgg_embedding = vgg_model.predict(vgg_array).flatten()
        
        # Concatenate embeddings
        combined_embedding = np.concatenate([ibed_embedding, vgg_embedding])
        embeddings.append(combined_embedding)
    
    return np.vstack(embeddings)

def get_bbox_region(image, bbox):
    if bbox.ndim > 1:
        bbox = bbox.flatten()
    bbox = np.round(bbox).astype(int)
    x1, y1, x2, y2 = bbox
    return image[y1:y2, x1:x2]

embeddings = []
embeddings_dict = {}
for i, result in enumerate(results):
    current_boxes = result.boxes.data.clone()
    orig_img = result.orig_img
    for box_num, box in enumerate(current_boxes):
        bbox_region = get_bbox_region(orig_img, box.numpy()[:4])
        embeddings_dict[len(embeddings)] = (i,box_num)
        embeddings.append(generate_embeddings([bbox_region], ibed, vgg_model))

X = np.vstack(embeddings)

# Create embedding matrix of all cutouts

In [ ]:
import os 

result_file_lst = sorted([file for file in os.listdir('/Users/nickjohnson/olm_data/olm_pics/') if file.endswith('.jpg') 
        or file.endswith('.jpeg')
        or file.endswith('.png')
        or file.endswith('.JPG')])

In [ ]:
import numpy as np

# Load from .npy file
X = np.load('my_array.npy')

In [ ]:
import fcntl
import gc
from scipy.spatial.distance import cdist

def update_result(result, box_index, brand_id, txt_file):
    img_width, img_height = result.orig_shape[1], result.orig_shape[0]
    # Read the file
    updated_boxes = []
    if os.path.exists(txt_file):
        with open(txt_file, 'r') as f:
            # Lock the file exclusively
            fcntl.flock(f, fcntl.LOCK_EX)

            # Read the file content
            lines = f.readlines()
            fcntl.flock(f, fcntl.LOCK_UN)

        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  # Ensure the line is correctly formatted
                class_number = int(parts[0])
                x_center = float(parts[1]) * img_width
                y_center = float(parts[2]) * img_height
                width = float(parts[3]) * img_width
                height = float(parts[4]) * img_height
                
                # Convert from center coordinates to top-left and bottom-right
                x1 = x_center - width / 2
                y1 = y_center - height / 2
                x2 = x_center + width / 2
                y2 = y_center + height / 2
                
                # Create the tensor for this box, (x1, y1, x2, y2, confidence, class_id)
                box_tensor = torch.tensor([x1, y1, x2, y2, 1, class_number], dtype=torch.float32)
                updated_boxes.append(box_tensor)

        updated_boxes_tensor = torch.stack(updated_boxes)
        os.remove(txt_file)

    else:
        updated_boxes_tensor = result.boxes.data.clone()

    updated_boxes_tensor[box_index,-1] = brand_id
    result.update(boxes=updated_boxes_tensor)
    result.save_txt(txt_file)
    del updated_boxes
    del updated_boxes_tensor

brand_ids = {}
assignment_dict = {}
unassigned = []
src = '/Users/nickjohnson/olm_data/clustered_images/branded'
for i, brand in enumerate(sorted(f for f in os.listdir(src) if f != '.DS_Store')):
    gc.collect()
    path = os.path.join(src,brand)
    if os.path.isdir(path):
        brand_id = i + 1
        print(brand, brand_id)
        brand_ids[brand] = brand_id
        for image in os.listdir(path):
            if not image.endswith('.jpg') or image.endswith('.jpeg') or image.endswith('.png') or image.endswith('.JPG'):
                continue
            image_path = os.path.join(path, image)
            image_embedding = generate_embeddings([image_path], ibed, vgg_model)
            sims = cdist(image_embedding.reshape(1, -1), X)
            del image_embedding
            embedding_indicies = np.argsort(sims)
            for embedding_index in embedding_indicies[0]:
                result_indicies = embeddings_dict[embedding_index]
                if sims[0, embedding_index] < assignment_dict.get(result_indicies, (0, 999999999))[-1]:
                    result_index = result_indicies[0]
                    result_file = result_file_lst[result_index]
                    if result_indicies in assignment_dict:
                        unassigned.append(image_path)
                    assignment_dict[result_indicies] = (image_path, sims[0, embedding_index])
                    break
            else:
                unassigned.append(image_path)
            del sims
            result = model('/Users/nickjohnson/olm_data/olm_pics/' + result_file, conf=0.05, iou=0)[0]
            box_index = result_indicies[1]
            txt_file = f'./predictions_olm_pics/{result_file.split(".")[0]}.txt'
            update_result(result, box_index, brand_id, txt_file)
            del result
    
    print(len(unassigned))

# Sorting the dictionary by IDs (values)
sorted_brands = sorted(brand_ids.items(), key=lambda item: item[1])

# Writing the sorted brand names to classes.txt
with open("classes.txt", "w") as file:
    for brand, _ in sorted_brands:
        file.write(f"{brand}\n")

100_grand 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step

image 1/1 /Users/nickjohnson/olm_data/olm_pics/w6MqgfrvFMrh5dR6soTyw3jo1m3VEI4uCbP67hI2.jpeg: 640x480 1 logo, 676.7ms
Speed: 2.8ms preprocess, 676.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)
0
1800 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step

image 1/1 /Users/nickjohnson/olm_data/olm_pics/05fOnfB6GdKclofkAjUlOan1VlYvQDGwUDPQ6HX5.jpeg: 480x640 3 logos, 576.1ms
Speed: 2.3ms preprocess, 576.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step

image 1/1 /Users/nickjohnson/olm_data/olm_pics/05fOnfB6GdKclofkAjUlOan1VlYvQDGwUDPQ6HX5.jpeg: 480x640 3 logos, 593.2ms
Speed: 3.0ms preprocess, 593.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
0
3_musketeers 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step

image 1/1 /Users/nickjohnson/olm_data/olm_pics/IhFG5HJ4HewsOS42nUGkWkpD9zOoJ4qRnBDjAyVI.jpg: 480x640 5 logos, 581.0ms
Speed: 3.6ms preprocess, 581.0ms i

In [ ]:
for image in unassigned:
    brand = image.split('/')[-2]
    brand_id = brand_ids[brand]
    if not image.endswith('.jpg') or image.endswith('.jpeg') or image.endswith('.png') or image.endswith('.JPG'):
                    continue
    image_path = image
    image_embedding = generate_embeddings([image_path], ibed, vgg_model)
    sims = cdist(image_embedding.reshape(1, -1), X)
    del image_embedding
    embedding_indicies = np.argsort(sims)
    for embedding_index in embedding_indicies[0]:
        result_indicies = embeddings_dict[embedding_index]
        if sims[0, embedding_index] < assignment_dict.get(result_indicies, (0, 999999999))[-1]:
            result_index = result_indicies[0]
            result_file = result_file_lst[result_index]
            if result_indicies in assignment_dict:
                unassigned.append(assignment_dict[result_indicies][0])
            assignment_dict[result_indicies] = (image_path, sims[0, embedding_index])
            break
    else:
        unassigned.append(image_path)
    del sims
    result = model('/Users/nickjohnson/olm_data/olm_pics/' + result_file, conf=0.05, iou=0)[0]
    box_index = result_indicies[1]
    txt_file = f'./predictions_olm_pics/{result_file.split(".")[0]}.txt'
    update_result(result, box_index, brand_id, txt_file)
    del result

['/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/im.jpg8627.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/im.jpg949.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/ordered_788_ordered_151_im.jpg9521.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/ordered_2165_ordered_2876_im.jpg9530.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/ordered_2046_ordered_2570_im.jpg9174.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/im.jpg8135.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/im.jpg8848.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/ordered_2110_ordered_2867_im.jpg1530.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/ordered_2166_ordered_2875_im.jpg9519.jpg',
 '/Users/nickjohnson/olm_data/clustered_images/branded/bud_light/ordered_2139_ordered_2782_im.jpg7543.jpg',
 '/Users/nickjohnson/olm_da

# Delete predictions that didn't have a match and reindex to 0

In [ ]:
import os

# Path to the directory containing the .txt files
directory = './predictions_olm_pics/'

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Construct full file path
        file_path = os.path.join(directory, filename)

        # Read the contents from the file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Process the lines
        processed_lines = []
        for line in lines:
            elements = line.split()
            first_element = int(elements[0])
            
            if first_element == 0:
                continue

            elements[0] = str(first_element - 1)
            processed_lines.append(' '.join(elements))

        # Write the processed data back to the same file
        with open(file_path, 'w') as file:
            for line in processed_lines:
                file.write(line + '\n')

Below renames all the files to the new format and copies missing images to the rescaled dir

In [ ]:
from collections import defaultdict
import os

# Create a defaultdict with list as the default value type
original_file_names = defaultdict(list)

# Directory path
directory_path = '/Users/nickjohnson/olm_data/rescaled/'

# Populate the dictionary, appending each file to the list of its corresponding key
for f in os.listdir(directory_path):
    key = f.split('_')[-1].split('.')[0]
    if key.isdigit():
        key = 'IMG_' + key
    original_file_names[key].append(f)

In [ ]:
full_scale_dict = {f.split('.')[0]:f for f in os.listdir('/Users/nickjohnson/olm_data/olm_pics/')}

In [ ]:
annotations_lst = [f.split('.')[0] for f in os.listdir('./predictions_olm_pics/')]

# Now we need to map the rescaled image names to the full scale image names

In [ ]:
from PIL import Image

#Let's go through each txt file
for image_name in annotations_lst:
    if image_name == 'classes' or image_name == '':
        continue
    #look for the possible names from the rescaled dir
    rescaled_image_names = original_file_names[image_name]
    full_scale_image_path = os.path.join('/Users/nickjohnson/olm_data/olm_pics/', full_scale_dict[image_name])
    annotatation_path = os.path.join('./predictions_olm_pics/', image_name + '.txt')
    max_sim = 0
    #if there's more than one, we need to find the one with the highest similarity
    if len(rescaled_image_names) > 1:
        full_scale_embedding = generate_embeddings([full_scale_image_path], ibed, vgg_model)
        for file in rescaled_image_names:
            rescaled_image_path = os.path.join('/Users/nickjohnson/olm_data/rescaled/', file)
            rescaled_embedding = generate_embeddings([rescaled_image_path], ibed, vgg_model)
            cos_sim = cosine_similarity(full_scale_embedding.reshape(1, -1), rescaled_embedding.reshape(1, -1))[0][0]
            if cos_sim > max_sim:
                max_sim = cos_sim
                max_path = rescaled_image_path
                max_file = file
        os.rename(annotatation_path, './predictions_olm_pics/' +  max_file.split('.')[0] + '.txt')
    #if there's only one, we can just rename it
    else:
        #unless, there are none. In that case, we need to look for the file
        if len(rescaled_image_names) == 0:
            for f in os.listdir('/Users/nickjohnson/olm_data/rescaled/'):
                #often, the org file had a '_' so our simple split will not work
                #we can try a more complex split
                possible_file_name = '_'.join(f.split('_')[-2:]).split('.')[0]
                if possible_file_name == image_name:
                    rescaled_image_names.append(f.split('.')[0] + '.txt')
                #if that doesn't work, if means the image does not exist in the rescaled dir
                #we need to look for the file in the 'scaled' dir
            if len(rescaled_image_names) == 0:
                for dirpath, dirnames, files in os.walk('/Users/nickjohnson/olm_data/scaled'):
                    for file in files:
                        if file.split('.')[0] == image_name:
                            relative_path = os.path.join(dirpath, image_name).split('scaled/', 1)[1]
                            transformed_path = relative_path.replace('/', '_')
                            rescaled_image_names.append(transformed_path + '.txt')
                            #since the image does not exist, we need to move it from scaled to rescaled with the new name format (this is mv the img, not rename txt)
                            os.rename(os.path.join(dirpath, file), '/Users/nickjohnson/olm_data/rescaled/' + transformed_path + '.' + file.split('.')[-1])
                            print('moving image:', os.path.join(dirpath, file), '/Users/nickjohnson/olm_data/rescaled/' + transformed_path + '.' + file.split('.')[-1])
                            break
                    
        os.rename(annotatation_path, './predictions_olm_pics/' +  rescaled_image_names[0].split('.')[0] + '.txt')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
moving image: /Users/nickjohnson/olm_data/scaled/2021/12/13/mUiI

This moves the images from the rescales dir to the same dir as the annotations

In [ ]:
import os
import shutil

# Define paths
annotations_dir = './predictions_olm_pics'  # Path where the annotations are stored
rescaled_images_dir = '/Users/nickjohnson/olm_data/rescaled'
destination_images_dir = './predictions_olm_pics'

# Ensure the destination directory exists
os.makedirs(destination_images_dir, exist_ok=True)

# Loop through all the txt files in the annotations directory
for file in os.listdir(annotations_dir):
    if file.endswith('.txt') and file not in ('classes.txt', '.DS_Store'):
        base_name = file[:-4]  # Remove the .txt extension to get the base name

        # Find corresponding image file in rescaled directory
        found = False
        for rescaled_file in os.listdir(rescaled_images_dir):
            if rescaled_file.startswith(base_name):
                # Determine full path of the source and destination
                rescaled_image_path = os.path.join(rescaled_images_dir, rescaled_file)
                destination_image_path = os.path.join(destination_images_dir, rescaled_file)

                # Copy file to the destination directory
                shutil.copy(rescaled_image_path, destination_image_path)
                found = True
                break

        if not found:
            raise FileNotFoundError(f"No matching rescaled image found for annotation {file}")

# After some editing in roboflow, we can remove images with no annotations

In [203]:
import os
import numpy as np

base_path = '/Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/labels'
image_path = '/Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/images'

def calculate_iou(box1, box2):
    # Convert the box values from strings to floats and calculate the intersection over union
    x1, y1, w1, h1 = [float(value) for value in box1]
    x2, y2, w2, h2 = [float(value) for value in box2]

    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1 - w1 / 2, x2 - w2 / 2)
    y_top = max(y1 - h1 / 2, y2 - h2 / 2)
    x_right = min(x1 + w1 / 2, x2 + w2 / 2)
    y_bottom = min(y1 + h1 / 2, y2 + h2 / 2)

    # Calculate the area of the intersection rectangle
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate and return the intersection over union (IoU)
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

def remove_overlapping_boxes(lines, threshold=0.8):
    boxes = [line.strip().split() for line in lines]  # Parse each line into parts
    remove_indices = []

    # Loop through each pair of boxes to calculate IoU and determine overlaps
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            if calculate_iou(boxes[i][1:], boxes[j][1:]) > threshold:
                # Add index to remove list, choosing between current and compared box based on class ID
                remove_indices.append(j if int(boxes[i][0]) == 218 else i)

    # Filter out the indices marked for removal and return the remaining lines
    return [line for idx, line in enumerate(lines) if idx not in set(remove_indices)]

# First pass: count occurrences of each class
class_count = {}
for file in os.listdir(base_path):
    if file.endswith('.txt'):
        with open(os.path.join(base_path, file), 'r') as f:
            for line in f:
                class_id = int(line.split()[0])
                if class_id in class_count:
                    class_count[class_id] += 1
                else:
                    class_count[class_id] = 1

# Second pass: filter and potentially reclassify annotations
for file in os.listdir(base_path):
    if file.endswith('.txt'):
        txt_file_path = os.path.join(base_path, file)
        with open(txt_file_path, 'r') as f:
            lines = f.readlines()
            filtered_lines = remove_overlapping_boxes(lines)

        if not filtered_lines:  # If there are no remaining lines, consider deleting the image
            os.remove(txt_file_path)
            image_file_path = os.path.join(image_path, file.replace('.txt', '.jpg'))
            if os.path.exists(image_file_path):
                os.remove(image_file_path)
                print(f"Deleted empty annotation and corresponding image: {image_file_path}")
        else:
            with open(txt_file_path, 'w') as f:
                for line in filtered_lines:
                    parts = line.split()
                    class_id = int(parts[0])
                    if class_count.get(class_id, 0) < 3:
                        parts[0] = '120'  # Reclassify as class 218
                    f.write(' '.join(parts) + '\n')

Deleted empty annotation and corresponding image: /Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/images/2022_04_09_tSmhmAAOGAA0lIrv93EKCtNdEFdjBnOE16ImDrcP_jpg.rf.86733154ba169752151bd5da8608ceb8.jpg
Deleted empty annotation and corresponding image: /Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/images/2022_06_06_C9a0QqRfPdH3aN7TwvdVX8ksOb7U05VbGUNe8yzm_jpg.rf.3b6a3a18d6cf0ea233604bad1724ffbf.jpg
Deleted empty annotation and corresponding image: /Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/images/2022_01_17_K0NRG2VfwZo2D0oDRK7JlX5qvfKLSEx48yaCSHeM_jpg.rf.d9d996208e5dfe618e3907bd9efb0f9e.jpg
Deleted empty annotation and corresponding image: /Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/images/2022_05_25_izHXCTdaEXdHmozIl4WTuET0frnleMoez1avshg1_jpg.rf.0baddbdcff4b4d8f1a4d82b99c9b1256.jpg
Deleted empty annotation and corresponding image: /Users/nickjohnson/downloads/Capstone.v1i.yolov8/train/images/2021_04_21_IMG_3006_jpg.rf.67ed57fc7ce2eac80a4ad4894be45

# Let's also rename the roboflow format back to the original format

In [8]:
import os

# Directories containing the images and labels
image_dir = '/Users/nickjohnson/downloads/Capstone_OLM_Logo_Recognition_Final.v2i.yolov8/train/images'
label_dir = '/Users/nickjohnson/downloads/Capstone_OLM_Logo_Recognition_Final.v2i.yolov8/train/labels'

# Function to rename files in a directory
def rename_files(directory):
    for filename in os.listdir(directory):
        # Determine where to cut off based on the first occurrence of _jpeg or _jpg
        index_jpeg = filename.find('_jpeg')
        index_jpg = filename.find('_jpg')
        
        # Find the minimum index that is not -1 (i.e., find which occurs first)
        if index_jpeg == -1 and index_jpg == -1:
            continue  # No match found, skip renaming
        if index_jpeg == -1:
            cut_index = index_jpg
        elif index_jpg == -1:
            cut_index = index_jpeg
        else:
            cut_index = min(index_jpeg, index_jpg)
        
        new_name = filename[:cut_index]  # New filename cutting off the undesired part
        new_name += '.jpg' if directory == image_dir else '.txt'  # Preserve the correct extension
        
        # Rename the file
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_name))

# Rename image files
rename_files(image_dir)

# Rename label files
rename_files(label_dir)

print("Files have been renamed.")

Files have been renamed.


# Finally, instead of rescaling the whole image we can crop first to preserve more fidelity in our area of interest

In [14]:
test_classes = ['amstel_light', 'aquafina', 'big_gulp', 'blue_moon', 'bodyarmor', 'brisk', 'bud_light', 'budweiser', 'burger_king', 'busch_light', 'canada_dry', 'caprisun', 'cheetos', 'cheezit', 'coca_cola', 'coors', 'corona', 'dasani', 'deer_park', 'doritos', 'dos_equis', 'dr_pepper', 'dunkin', 'fanta', 'fireball', 'fritos', 'funyuns', 'gatorade', 'great_value', 'guinness', 'heineken', 'heinz', 'hill_country_fare', 'jarritos', 'kirkland', 'kitkat', 'koolaid', 'lays', 'marlboro', 'mccafe', 'mcdonalds', 'michelob', 'mikes_hard', 'miller_light', 'milwaukees_best', 'modelo', 'monster', 'mot', 'mountain_dew', 'natural_ice', 'natural_light', 'nestle_pure_life', 'newport', 'niagara', 'nos', 'ozarka', 'panda_express', 'pepsi', 'philip_morris', 'poland_spring', 'popeyes', 'powerade', 'quick_trip', 'red_bull', 'reeses', 'ruffles', 'seven_eleven', 'snickers', 'solo', 'sprite', 'starbucks', 'starburst', 'stella', 'target', 'tropicana', 'truly', 'wawa', 'welchs', 'wendys', 'white_claw']
capestone_classes = ['amstel_light', 'aquafina', 'big_gulp', 'blue_moon', 'bodyarmor', 'brisk', 'bud_light', 'budweiser', 'burger_king', 'busch_light', 'canada_dry', 'caprisun', 'cheetos', 'cheezit', 'coca_cola', 'coors', 'corona', 'dasani', 'deer_park', 'doritos', 'dos_equis', 'dr_pepper', 'dunkin', 'fanta', 'fireball', 'fritos', 'funyuns', 'gatorade', 'great_value', 'guinness', 'heineken', 'heinz', 'hill_country_fare', 'jarritos', 'kirkland', 'kitkat', 'koolaid', 'lays', 'marlboro', 'mccafe', 'mcdonalds', 'michelob', 'mikes_hard', 'miller_light', 'milwaukees_best', 'modelo', 'monster', 'mountain_dew', 'natural_ice', 'natural_light', 'nestle_pure_life', 'newport', 'niagara', 'nos', 'ozarka', 'panda_express', 'pepsi', 'philip_morris', 'poland_spring', 'popeyes', 'powerade', 'quick_trip', 'red_bull', 'reeses', 'ruffles', 'seven_eleven', 'snickers', 'solo', 'sprite', 'starbucks', 'starburst', 'stella', 'target', 'tropicana', 'truly', 'wawa', 'welchs', 'wendys', 'white_claw']

In [15]:
my_dict = {}
for idx, cls in enumerate(test_classes):
    try:
        print(capestone_classes.index(cls))
        my_dict[idx] = capestone_classes.index(cls)
    except:
        print(cls)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
mot
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


In [21]:
my_dict[47]

KeyError: 47

In [22]:
import os
import boto3
import imageio.v3 as iio
from PIL import Image, ExifTags
import shutil

# AWS S3 setup
s3 = boto3.client('s3')
bucket_name = 'olm-pics-s3'

image_dir = '/Users/nickjohnson/downloads/Capstone_OLM_Logo_Recognition_Final.v2i.yolov8/train/images'
label_dir = '/Users/nickjohnson/downloads/Test.v1i.yolov8/train/labels'

def convert_heic_to_jpeg(heic_path, target_path):
    img = iio.imread(heic_path)
    image = Image.fromarray(img)
    image.save(target_path, format="JPEG")
    os.remove(heic_path)  # Clean up the original HEIC file

def download_and_convert_image(filename):
    path_elements = filename.split('_')
    for i in range(1, len(path_elements)):
        s3_path = '/'.join(path_elements[:i])
        file_prefix = '_'.join(path_elements[i:]).replace('.jpg', '')  # Remove .jpg to match against any extension

        # List objects in the directory
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_path)
        file_found = False
        local_filename_with_extension = os.path.join(image_dir, filename)  # Preserving the original filename for the download path

        if 'Contents' in response:
            for obj in response['Contents']:
                file_key = obj['Key']
                file_key_name = file_key.split('/')[-1].split('.')[0]  # Remove path and extension
                if file_prefix == file_key_name:
                    # Download the file if the prefix matches
                    try:
                        s3.download_file(bucket_name, file_key, local_filename_with_extension)
                        file_found = True
                        if local_filename_with_extension.lower().endswith('.heic'):
                            convert_heic_to_jpeg(local_filename_with_extension, local_filename_with_extension.replace('.HEIC', '.jpg').replace('.heic', '.jpg'))
                        break
                    except Exception as e:
                        continue
            if file_found:
                break

    if not file_found:
        print(f"File not found in S3 for any expected extensions or variations: {file_prefix}")

    return local_filename_with_extension.replace('.HEIC', '.jpg').replace('.heic', '.jpg')  # Ensure the local path is for a JPEG file

def rotate_image_based_on_exif(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = dict(img._getexif().items())

        if exif[orientation] == 3:
            img = img.rotate(180, expand=True)
        elif exif[orientation] == 6:
            img = img.rotate(270, expand=True)
        elif exif[orientation] == 8:
            img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # cases: image don't have getexif
        pass
    return img

def process_image(image_path, label_path):
    with Image.open(image_path) as img:
        img = rotate_image_based_on_exif(img)
        original_width, original_height = img.size

        with open(label_path, 'r') as file:
            boxes = [list(map(float, line.strip().split())) for line in file.readlines()]
            if len(boxes) == 0:
                return
            for box in boxes:
                if int(box[0]) == 47:
                    return
                
        abs_boxes = [
            [box[0],  # class_id as float from file
             box[1] * original_width - 0.5 * box[3] * original_width,  # xmin
             box[2] * original_height - 0.5 * box[4] * original_height,  # ymin
             box[1] * original_width + 0.5 * box[3] * original_width,  # xmax
             box[2] * original_height + 0.5 * box[4] * original_height]  # ymax
            for box in boxes]

        # Calculate distances from the edges
        min_xmin_distance = min(box[1] for box in abs_boxes)
        min_ymin_distance = min(box[2] for box in abs_boxes)
        max_xmax_distance = max(box[3] for box in abs_boxes)
        max_ymax_distance = max(box[4] for box in abs_boxes)

        xmin_distance = min_xmin_distance
        ymin_distance = min_ymin_distance
        xmax_distance = original_width - max_xmax_distance
        ymax_distance = original_height - max_ymax_distance

        min_edge_distance = min(xmin_distance, ymin_distance, xmax_distance, ymax_distance)

        # Check if the closest bounding box to any edge is too close
        buffer_threshold = 0.05  # 5% of the image dimension
        if min(xmin_distance/original_width, ymin_distance/original_height, xmax_distance/original_width, ymax_distance/original_height) < buffer_threshold:
            # If too close, do not crop
            img_resized = img.resize((640, int(original_height * (640 / original_width))), Image.Resampling.LANCZOS)
            new_boxes = [[int(box[0]), box[1], box[2], box[3], box[4]] for box in boxes]
        else:
            # Apply cropping logic
            if len(boxes) == 1:
                buffer = min_edge_distance * 0.7
            elif len(boxes) == 2:
                buffer = min_edge_distance * 0.4
            else:
                buffer = min_edge_distance * 0.2
                
            xmin = max(min_xmin_distance - buffer, 0)
            ymin = max(min_ymin_distance - buffer, 0)
            xmax = min(max_xmax_distance + buffer, original_width)
            ymax = min(max_ymax_distance + buffer, original_height)

            img_cropped = img.crop((xmin, ymin, xmax, ymax))
            cropped_width, cropped_height = img_cropped.size
            img_resized = img_cropped.resize((640, int(cropped_height * (640 / cropped_width))), Image.Resampling.LANCZOS)
            new_boxes = [
            [
                int(my_dict[int(box[0])]),  # class_id
                (box[1] * original_width - xmin) / cropped_width,  # new x_center
                (box[2] * original_height - ymin) / cropped_height,  # new y_center
                box[3] * original_width / cropped_width,  # new width normalized
                box[4] * original_height / cropped_height  # new height normalized
            ]
            for box in boxes]

        img_resized.save(image_path, format='JPEG')
        with open(label_path, 'w') as file:
            for box in new_boxes:
                file.write(' '.join(map(str, [box[0]] + box[1:])) + '\n')

for annotation in os.listdir(label_dir):
    if not annotation.endswith('.txt') and annotation != 'classes.txt':
        continue

    image_file = annotation.replace('.txt', '.jpg')

    if not annotation.split('.')[0] in [image.split('.')[0] for image in os.listdir(image_dir)]:
        print(f"Processing {image_file}...")
        local_image_path = download_and_convert_image(image_file)
        local_label_path = os.path.join(label_dir, image_file.replace('.jpg', '.txt'))
        process_image(local_image_path, local_label_path)
        #move annotation file from Test.v1i.yolov8 to Capstone_OLM_Logo_Recognition_Final.v2i.yolov8
        shutil.move(os.path.join(label_dir, annotation), os.path.join('/Users/nickjohnson/downloads/Capstone_OLM_Logo_Recognition_Final.v2i.yolov8/train/labels', annotation))

Processing 2023_03_30_NY402Cee3QeRzDGnsX2qcuIg5j4oW6msMGXqiWU2.jpg...
Processing 2022_02_27_jcYJ2s2N4w3biYLARbJ7TkdRdSrhBqp1d7XW37YF.jpg...
Processing 2018_08_24_m4gaF0eRTvJzpjf0CbUQd7hJ8Z8gRuV6MOkiD2MF.jpg...
Processing 2019_09_11_G0zt8hEZWtJ0Zw70eLGKpb8M776NyFrjsfUlMpak.jpg...
Processing 2022_02_27_v2OMizi4jI9tFiYroA6trPSIQwhOZsSV5WTwhzef.jpg...
Processing 2020_01_12_uUSnsSp9PuFM3P0ow60oWTRlmsXy2XqjIyma27vf.jpg...
Processing 2022_03_30_niRCvTHTEarjgd68ePbW0TS2woJ5qcGYootOcAY0.jpg...
Processing 2023_03_30_wEmV8Z3SlaKTgvxksBxZ3UAQCGi7D53fI2i382yS.jpg...
Processing 2019_03_03_v32r2OxVk9IzHMn8aC3Z25YVO0gt9LOiYcy8LEHl.jpg...
Processing 2022_02_25_g4pw2Bqx5rFWHfsGVjtFB1j0jtFcaKZMbFEjWlhW.jpg...
Processing 2022_07_19_aLSVO9Sr60or2v2HUNvWHl9qHzDcjJhGMnuky4rl.jpg...
Processing 2022_02_22_fvjryK4n0w1c4VNPFm5O8DQsdaCwcLKNpKRIuKQA.jpg...
Processing 2022_06_11_1kiro1i9p66vOOUKsbHxY52h4H5xQIkwf6m1v3HE.jpg...
Processing 2020_07_25_IMG_1848.jpg...
Processing 2021_10_14_rX0VSsRDUDjxYeNCgeqDEPDQJQ7MBZ

In [ ]:
from ultralytics import YOLO

model = YOLO('./generic_logo_model/detect/train/weights/best.pt')

# Train the model with 2 GPUs
results = model.train(data="./Capstone_OLM_Logo_Recognition.v3i.yolov8/data.yaml", epochs=1, batch = 4, imgsz=640, device="mps")